In [209]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from matplotlib import pyplot
import pickle
import csv

In [210]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [211]:
def get_pass_streak(y_project):
    p = y_project[0]
    pass_streak = [y_project[0]]
    for i in range(1, len(y_project)):
        if y_project[i] == 1:
            p += 1
        else:
            p = 0
        pass_streak.append(p)
    return pass_streak

In [212]:
cloud_controller = pd.read_csv('metrics_data/cloud_controller_ng_metrics.csv')

with open('../cloud_controller_ng_indexes.pkl', 'rb') as load_file:
    cloud_train_build_ids = pickle.load(load_file)
    cloud_test_build_ids = pickle.load(load_file)

cloud_controller = cloud_controller [ cloud_controller['tr_build_id'].isin(cloud_train_build_ids)]
cloud_train_build_ids = cloud_controller['tr_build_id'].tolist()
res_cloud_controller = pd.read_csv('../data/cloud_controller_ng.csv', usecols = ['tr_build_id', 'tr_status'])
y_cloud_controller = res_cloud_controller [ res_cloud_controller['tr_build_id'].isin(cloud_train_build_ids)]['tr_status'].tolist()
y_cloud_controller = output_values(y_cloud_controller)
cloud_controller['tr_status'] = y_cloud_controller

In [213]:
geoserver = pd.read_csv('metrics_data/geoserver_metrics.csv')

with open('../geoserver_indexes.pkl', 'rb') as load_file:
    geoserver_train_build_ids = pickle.load(load_file)
    geoserver_test_build_ids = pickle.load(load_file)

geoserver = geoserver [ geoserver['tr_build_id'].isin(geoserver_train_build_ids)]
geoserver_train_build_ids = geoserver['tr_build_id'].tolist()
res_geoserver = pd.read_csv('../data/geoserver.csv', usecols = ['tr_build_id', 'tr_status'])
y_geoserver = res_geoserver [ res_geoserver['tr_build_id'].isin(geoserver_train_build_ids)]['tr_status'].tolist()
y_geoserver = output_values(y_geoserver)
geoserver['tr_status'] = y_geoserver

In [214]:
gradle = pd.read_csv('metrics_data/gradle_metrics copy 2.csv')

with open('../gradle_indexes.pkl', 'rb') as load_file:
    gradle_train_build_ids = pickle.load(load_file)
    gradle_test_build_ids = pickle.load(load_file)

gradle = gradle [ gradle['tr_build_id'].isin(gradle_train_build_ids)]
gradle_train_build_ids = gradle['tr_build_id'].tolist()
res_gradle = pd.read_csv('../data/gradle.csv', usecols = ['tr_build_id', 'tr_status'])
y_gradle = res_gradle [ res_gradle['tr_build_id'].isin(gradle_train_build_ids)]['tr_status'].tolist()
y_gradle = output_values(y_gradle)
gradle['tr_status'] = y_gradle

In [215]:
projects = [gradle, cloud_controller, geoserver]

In [216]:
gradle.drop('num_commits', inplace=True, axis=1)
gradle.drop('reviewer_experience', inplace=True, axis=1)
gradle.drop('num_of_reviewers', inplace=True, axis=1)

cloud_controller.drop('num_commits', inplace=True, axis=1)
cloud_controller.drop('reviewer_experience', inplace=True, axis=1)
cloud_controller.drop('num_of_reviewers', inplace=True, axis=1)

geoserver.drop('num_commits', inplace=True, axis=1)
geoserver.drop('reviewer_experience', inplace=True, axis=1)
geoserver.drop('num_of_reviewers', inplace=True, axis=1)

In [217]:
gradle['num_of_passes'] = get_pass_streak(y_gradle)
cloud_controller['num_of_passes'] = get_pass_streak(y_cloud_controller)
geoserver['num_of_passes'] = get_pass_streak(y_geoserver)

In [218]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

In [219]:
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth
             }

In [220]:
forest = RandomForestClassifier()
grid_search = GridSearchCV(estimator = forest, param_grid = param_grid, 
                      cv = 3, n_jobs = -1, verbose = 2)

In [221]:
#estimating for geoserver

X_train = geoserver
y_train = y_geoserver

geoserver = pd.read_csv('metrics_data/geoserver_metrics.csv')
X_test = np.array(geoserver [ geoserver['tr_build_id'].isin(geoserver_test_build_ids)])
geoserver_train_build_ids = geoserver['tr_build_id'].tolist()
res_geoserver = pd.read_csv('../data/geoserver.csv', usecols = ['tr_build_id', 'tr_status'])
y_geoserver = res_geoserver [ res_geoserver['tr_build_id'].isin(geoserver_test_build_ids)]['tr_status'].tolist()
y_test = np.array(output_values(y_geoserver))

In [246]:
print(X_train)

      tr_build_id  patch_size  num_of_comments  test_file_changes  \
0        11905123           0                0                  0   
1        11905554           0                0                  0   
2        11909050           0                0                 31   
3        11939345         397                0                 21   
4        11939991           0                0                  0   
...           ...         ...              ...                ...   
2085    146183118          37                0                  0   
2086    146593522          13                0                 15   
2087    146603882           0                0                  0   
2088    146629589          43                0                 39   
2089    146642184           9                0                104   

      files_added  files_deleted  frequency_file_change  developer_experience  \
0               0              0                      1                     0   
1        

In [224]:
sample_size = int(len(geoserver)*0.7)

best_precision = 0
best_recall = 0
best_accuracy = 0
best_threshold = 0

best_precision_sample = np.array([0])
best_precision_sample_result = np.array([0])
best_recall_sample = np.array([0])
best_recall_sample_result = np.array([0])
best_accuracy_sample = np.array([0])
best_accuracy_sample_result = np.array([0])

best_precision_estimator = 0
best_recall_estimator = 0
best_accuracy_estimator = 0

for i in range(100):
    
    sample_train = resample(X_train, replace=True, n_samples=sample_size)
    sample_train_result = np.array(sample_train['tr_status'])
    
    build_ids = sample_train['tr_build_id'].tolist()
    
    sample_test = X_train [~X_train['tr_build_id'].isin(build_ids)]
    sample_test_result = np.array(sample_test['tr_status'])
    
    print(len(sample_train))
    print(len(sample_test))
    
    
    sample_train.drop('tr_status', inplace=True, axis=1)
    sample_train.drop('tr_build_id', inplace=True, axis=1)
    sample_test.drop('tr_status', inplace=True, axis=1)
    sample_test.drop('tr_build_id', inplace=True, axis=1)
    
    
    sample_train = np.array(sample_train)
    sample_test = np.array(sample_test)
    
    grid_search.fit(sample_train, sample_train_result)
    sample_pred_result = grid_search.predict_proba(sample_test)
    #print(sample_pred_result)
    
    pred_vals = sample_pred_result[:, 1]
    #print(pred_vals)
    fpr, tpr, thresholds = roc_curve(sample_test_result, pred_vals)
    gmeans = sqrt(tpr * (1-fpr))
    ix = argmax(gmeans)
    best_threshold = thresholds[ix]
    
    final_pred_result = []
    for i in range(len(pred_vals)):
        if pred_vals[i] > best_threshold:
            final_pred_result.append(1)
        else:
            final_pred_result.append(0)
    
    accuracy = accuracy_score(sample_test_result, final_pred_result)
    precision = precision_score(sample_test_result, final_pred_result)
    recall = recall_score(sample_test_result, final_pred_result)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_accuracy_sample = sample_train
        best_accuracy_sample_result = sample_train_result
        best_accuracy_estimator = grid_search.best_estimator_
        
    if precision > best_precision:
        best_precision = precision
        best_precision_sample = sample_train
        best_precision_sample_result = sample_train_result
        best_precision_estimator = grid_search.best_estimator_
        best_threshold = thresholds[ix]
        
    if recall > best_recall:
        best_recall = recall
        best_recall_sample = sample_train
        best_recall_sample_result = sample_train_result
        best_recall_estimator = grid_search.best_estimator_
    
    print(precision, recall, accuracy)
        
    

1463
914
Fitting 3 folds for each of 60 candidates, totalling 180 fits


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1.0 0.997867803837953 0.9989059080962801
1463
903
Fitting 3 folds for each of 60 candidates, totalling 180 fits


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1.0 0.9978632478632479 0.9988925802879292
1463
931
Fitting 3 folds for each of 60 candidates, totalling 180 fits


...

1.0 0.9979508196721312 0.9989023051591658
1463
927
Fitting 3 folds for each of 60 candidates, totalling 180 fits


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1.0 0.9978768577494692 0.9989212513484358
1463
923
Fitting 3 folds for each of 60 candidates, totalling 180 fits


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1.0 0.9978448275862069 0.9989165763813651


In [232]:
geoserver = pd.read_csv('metrics_data/geoserver_metrics.csv')
geoserver.drop('num_commits', inplace=True, axis=1)
geoserver.drop('reviewer_experience', inplace=True, axis=1)
geoserver.drop('num_of_reviewers', inplace=True, axis=1)


X_test = geoserver [ geoserver['tr_build_id'].isin(geoserver_test_build_ids)]
geoserver_train_build_ids = geoserver['tr_build_id'].tolist()
X_test.drop('tr_build_id', inplace=True, axis=1)
X_test.drop('tr_status', inplace=True, axis=1)
res_geoserver = pd.read_csv('../data/geoserver.csv', usecols = ['tr_build_id', 'tr_status'])
y_geoserver = res_geoserver [ res_geoserver['tr_build_id'].isin(geoserver_test_build_ids)]['tr_status'].tolist()
y_test = np.array(output_values(y_geoserver))

In [245]:
estimator = best_precision_estimator
estimator.fit(best_precision_sample, best_precision_sample_result)

print(best_threshold)
queue = 0
pred_results = []
for val in X_test:
    val = np.append(val, [queue])
    val = val.reshape(1, 9)
    pred = estimator.predict_proba(val)
    if pred[0][1] > best_threshold:
        pred_results.append(1)
        queue += 1
    else:
        pred_results.append(0)
        queue = 0

pred_results = np.array(pred_results)
precision_score(y_test, pred_results)

0.746


ValueError: X has 9 features, but RandomForestClassifier is expecting 8 features as input.

In [239]:
accuracy_score(y_test, pred_results)

0.5359477124183006

In [38]:
grid_search.fit(X_train, y_train)
y_pred_test = grid_search.predict(X_test)
accuracy_score(y_test, y_pred_test)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits


0.7177033492822966

In [64]:
queue = y_test[-10:].tolist()
max_queue_length = 10

y_pred_test = []
for index in range(len(X_test)):
    new_build = X_test[index]
    new_build[-1] = queue.count(1)
    new_build = new_build.reshape((1,9))
    predict_result = grid_search.predict(new_build)
    queue.pop(0)
    queue.append(predict_result[0])
    y_pred_test.append(predict_result[0])

In [65]:
accuracy_score(y_test, y_pred_test)

0.7129186602870813

In [66]:
print(y_test)

[1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1
 1 1 1 0 1 0 0 0 0 1 0]


In [67]:
print(y_pred_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [68]:
confusion_matrix(y_test, y_pred_test)

array([[  5, 113],
       [  7, 293]])